# Two Input Networks Using Categorical Embeddings, Shared Layers, and Merge Layers
We're gonna use a larger dataset to predict team strength. Each time has an integer ID.

In [2]:
import pandas as pd
import numpy as np
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Add
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
df = pd.read_csv('basketball_data/games_season.csv')
df.head()

,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1


## Category embeddings
Advanced type of layer. They are very useful in dealing with high cardinality categorical data like out team's ID. 

Also useful when using word to vector models.

We're gonna use a cat embedding layer to map integer team ID to a decimal rating. 

For a clearer overview of embedding layers:

https://towardsdatascience.com/deep-learning-4-embedding-layers-f9a02d55ac12

https://www.youtube.com/watch?v=qpb_39IjZA0

In [4]:
from keras.layers import Embedding
input_tensor = Input(shape=(1, ))
n_teams = 10887

embed_layer = Embedding(input_dim = n_teams,
                        input_length =1, #because each team is an integer
                        output_dim =1, #we only want a single rating per team
                        name = 'Team-Strength-Lookup')
embed_tensor = embed_layer(input_tensor)

Instructions for updating:
Colocations handled automatically by placer.


Embedding layers increase dimesionality of the data. This extra dimesion can be useful when dealing with images or text. We won't use this here so we will flatten it.

The flatten layer is also the output layer for the embedding process

In [5]:
from keras.layers import Flatten
flatten_tensor = Flatten()(embed_tensor)

We can wrap the whole thing in our model:

In [6]:
model = Model(input_tensor, flatten_tensor)

We can now use this model as a layer for another model. 

### Practice: Define team lookup
Shared layers allow a model to use the same weight matrix for multiple steps. In this exercise, you will build a "team strength" layer that represents each team by a single number. You will use this number for both teams in the model. The model will learn a number for each team that works well both when the team is team_1 and when the team is team_2 in the input data.

The games_season DataFrame is available in your workspace.

In [7]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(df['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

#The embedding layer is a lot like a dictionary, but your model learns the values for each key.

### Practice: Define team model
The team strength lookup has three components: an input, an embedding layer, and a flatten layer that creates the output.

If you wrap these three layers in a model with an input and output, you can re-use that stack of three layers at multiple places.

Note again that the weights for all three layers will be shared everywhere we use them.

In [8]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

#The model will be reusable, so you can use it in two places in your final model

## Shared Layers
We're creating a model with two inputs; one for each team in the basketball dataset. We want each team to use the SAME embedding layer we defined before. 

Shared lay. allow us to define an operation and apply the exact same op w/ exact same weights on diff inputs. 

In our model we'll share team rating for both inputs. The learned rating will be the same to whatever team it applies

![](shared.PNG)

In [9]:
#First we have to create 2 or more inputs
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))

shared_layer = Dense(1)
output_tensor_1 = shared_layer(input_tensor_1)
output_tensor_2 = shared_layer(input_tensor_2)

We can also share models, not just layers. We can then define modular components of models and then re-use them 

In [10]:
#First we have to create 2 or more inputs
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))
#using our model from last section
output_tensor_1 = model(input_tensor_1)
output_tensor_2 = model(input_tensor_2)

As far as I understood all this is to create a function in NN world. Callable for different inputs and gives diff outputs without modifying the weights everytime. 

### Practice: Defining two inputs
In this exercise, you will define two input layers for the two teams in your model. This allows you to specify later in the model how the data from each team will be used differently.

In [11]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input((1,), name = 'Team-1-In')

# Separate input layer for team 2
team_in_2 = Input((1,), name = 'Team-2-In')

### Practice: Lookup both inputs in the same model
Now that you have a team strength model and an input layer for each team, you can lookup the team inputs in the shared team strength model. The two inputs will share the same weights.

In this dataset, you have 10,888 unique teams. You want to learn a strength rating for each team, such that if any pair of teams plays each other, you can predict the score, even if those two teams have never played before. Furthermore, you want the strength rating to be the same, regardless of whether the team is the home team or the away team.

To achieve this, you use a shared layer, defined by the re-usable model (team_strength_model()) you built in exercise 3 and the two input layers (team_in_1 and team_in_2) from the previous exercise, all of which are available in your workspace.

In [14]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

## Merge layers
Now that we have multiple inputs and a shared layer we need to combine the inputs into a single layer that we can use to predict a single output. 

Merge layers allow us to define advanced non-sequential network topologies. We have diff options:
- add, subtract and multiply do elementary operations element-wise on the input layers and then require them to be have the same shape.
- we can also concatenate both layers together. Can operate on layers with diff size

In [15]:
#simple adding example
from keras.layers import Add
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))
#we could also add more than 2
output_tensor = Add()([input_tensor_1, input_tensor_2])

We can wrap the output of the add layer inside a model so we can use it to fit data later on.

In [17]:
model = Model([input_tensor_1, input_tensor_2], output_tensor)
#in order to fit data with it we have to compile it first.
model.compile('adam', 'mean_absolute_error')

### Practice: Output layer using shared layer
Now that you've looked up how "strong" each team is, subtract the team strengths to determine which team is expected to win the game.

This is a bit like the seeds that the tournament committee uses, which are also a measure of team strength. But rather than using seed differences to predict score differences, you'll use the difference of your own team strength model to predict score differences.

The subtract layer will combine the weights from the two layers by subtracting them.

In [18]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

![](subtractmodel.PNG)

In [19]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

## Predict from your model
Keras models with multiple inputs work exactly the same way as models with single input. Same fit, evaluate and predict. But now all of this methods will take a list of inputs. 

### Practice: Fit the model to the regular season training data
Now that you've defined a complete team strength model, you can fit it to the basketball data! Since your model has two inputs now, you need to pass the input data as a list.

In [21]:
# Get the team_1 column from the regular season data
input_1 = df['team_1']

# Get the team_2 column from the regular season data
input_2 = df['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2],
          df['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=0.1,
          verbose=True)
#Now our model has learned a strength rating for every team.



Instructions for updating:
Use tf.cast instead.
Train on 280960 samples, validate on 31218 samples
Epoch 1/1
280960/280960 [==============================] - 1s 4us/step - loss: 12.1201 - val_loss: 11.8381


### Practice: Evaluate the model on the tournament test data
The model you fit to the regular season data (model) in the previous exercise and the tournament dataset (games_tourney) are available in your workspace.

In this exercise, you will evaluate the model on this new dataset. This evaluation will tell you how well you can predict the tournament games, based on a model trained with the regular season data. This is interesting because many teams play each other in the tournament that did not play in the regular season, so this is a very good check that your model is not overfitting.

In [23]:
#game_season_tournament
#includes the seed difference
#we have 16 teams, the first 4 had a seed of 1 and the last 4 of 16

games_tourney = pd.read_csv('basketball_data/games_tourney.csv')
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [24]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
model.evaluate([input_1, input_2], games_tourney['score_diff'])

4234/4234 [==============================] - 0s 25us/step


11.681071017900052